[View in Colaboratory](https://colab.research.google.com/github/Yugandhartripathi/SemanticInpainting/blob/master/Semantic_Inpainting.ipynb)




# Semantic Inpainting

In [1]:
!ls

datalab


In [2]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()
import numpy as np
import tensorflow as tf
import os,time
from glob import glob

from ops import batch_norm,linear,conv2d,deconv2d,lrelu
from image_helpers import *

Saving image_helpers.py to image_helpers.py
Saving ops.py to ops.py


In [0]:
#Parameter Defintion
is_crop=True
batch_size=64
image_size=108
sample_size=64
image_shape=[64,64,3]

z_dim=100

gf_dim=64
df_dim=64

learning_rate=0.0002
beta1=0.5

In [0]:
#Batch Normalisation objects
d_bn1 = batch_norm(name='d_bn1')
d_bn2 = batch_norm(name='d_bn2')
d_bn3 = batch_norm(name='d_bn3')

g_bn0 = batch_norm(name='g_bn0')
g_bn1 = batch_norm(name='g_bn1')
g_bn2 = batch_norm(name='g_bn2')
g_bn3 = batch_norm(name='g_bn3')

In [0]:
def discriminator(image,reuse=False):
    if reuse:
        tf.get_variable_scope().reuse_variables()
        
    h0=lrelu(conv2d(image,df_dim,name='d_h0_conv'))
    h1=lrelu(d_bn1(conv2d(h0,df_dim*2,name='d_h1_conv')))
    h2=lrelu(d_bn2(conv2d(h1,df_dim*4,name='d_h2_conv')))
    h3=lrelu(d_bn3(conv2d(h2,df_dim*8,name='d_h3_conv')))
    h4=linear(tf.reshape(h3,[batch_size,-1]),1,'d_h3_lin')
             
    return tf.nn.sigmoid(h4),h4

In [0]:
def generator(z):
    z_=linear(z,gf_dim*8*4*4,'g_h0_lin')
    h0=tf.nn.relu(g_bn0(tf.reshape(z_,[-1,4,4,gf_dim*8])))
    h1=tf.nn.relu(g_bn1(deconv2d(h0,[batch_size,8,8,gf_dim*4],name='g_h1')))
    h2=tf.nn.relu(g_bn2(deconv2d(h1,[batch_size,16,16,gf_dim*2],name='g_h2')))
    h3=tf.nn.relu(g_bn3(deconv2d(h2,[batch_size,32,32,gf_dim*1],name='g_h3')))
    h4 = deconv2d(h3, [batch_size, 64, 64, 3], name='g_h4')
    
    return tf.nn.tanh(h4)

In [0]:
#Building model
images=tf.placeholder(tf.float32,[batch_size]+image_shape,name='real_images')
sample_images=tf.placeholder(tf.float32,[sample_size]+image_shape,name="sample_images")
z=tf.placeholder(tf.float32,[None,z_dim])

G=generator(z)
D,D_logits=discriminator(images)
D_,D_logits_=discriminator(G,reuse=True)

#cost fn
d_loss_real=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits,labels=tf.ones_like(D)))
d_loss_fake=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits_,labels=tf.zeros_like(D_)))
d_loss=d_loss_real+d_loss_fake

g_loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits_,labels=tf.ones_like(D_)))
#For image completion
mask=tf.placeholder(tf.float32,[None]+image_shape,name="mask")

contextual_loss=tf.reduce_sum(tf.contrib.layers.flatten(tf.abs(tf.multiply(mask,G)-tf.multiply(mask,images))))
perceptual_loss=g_loss
complete_loss=contextual_loss+perceptual_loss

In [0]:
#Optimizers
t_vars=tf.trainable_variables()
sess=tf.Session()
sess.run(tf.global_variables_initializer())

d_vars=[var for var in t_vars if 'd_' in var.name]
g_vars=[var for var in t_vars if 'g_' in var.name]

with tf.variable_scope(tf.get_variable_scope(),reuse=tf.AUTO_REUSE):
    d_optim = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5).minimize(d_loss, var_list=d_vars)
    g_optim = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5).minimize(g_loss, var_list=g_vars)
    complete_optim=tf.train.AdamOptimizer(learning_rate).minimize(complete_loss,var_list=g_vars)

## Data Points

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import io
import zipfile
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '17Cl4ZeCYGzONypZo1WPiIz20x06ZgPAt' #-- Updated File ID for my zip
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString(encoding='cp862')))
downloaded.GetContentFile('img_align_celeba.zip')


In [37]:
# UNZIP ZIP
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('img_align_celeba.zip', 'r')
zip_ref.extractall('MODEL/')
zip_ref.close()

Uncompressing zip file


In [0]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

saver=tf.train.Saver()

In [12]:
#DATASET 
data=glob(os.path.join('MODEL/img_align_celeba/','*.jpg'))

sample_z=np.random.uniform(-1,1,size=(sample_size,z_dim))
sample_files=data[0:sample_size]
sample=[get_image(sample_file,image_size,is_crop) for sample_file in sample_files]
sample_images=np.reshape(np.array(sample).astype(np.float32),[sample_size]+image_shape)


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


## Training the model

In [38]:
#Training
uploadId = '1TLTtsRW366oRUdbxjWk2afqCEXvMu_t5'
counter=1
start_time=time.time()
for epoch in range(20):
    np.random.shuffle(data)
    batchidxs=int(len(data)/batch_size)
    
    for idx in range(batchidxs):
        #try:
            batch_files=data[idx*batch_size:(idx+1)*batch_size]
            batch=[get_image(batch_file,image_size,is_crop=is_crop) for batch_file in batch_files]
            batch_images=np.reshape(np.array(batch).astype(np.float32),[batch_size]+image_shape)

            batch_z=np.random.uniform(-1,1,[batch_size,z_dim]).astype(np.float32)

            #mask
            scale=0.25
            mask_=np.ones([batch_size]+image_shape).astype(np.float32)
            l=int(64*scale)
            u=int(64*(1.0-scale))
            mask_[:,l:u,l:u,:]=0.0
            #inverse mask
            scale=0.25
            imask_=np.zeros([batch_size]+image_shape).astype(np.float32)
            l=int(64*scale)
            u=int(64*(1.0-scale))
            imask_[:,l:u,l:u,:]=1.0
            
            
            fd={z:batch_z,images:batch_images}
            sess.run([g_optim,d_optim],feed_dict=fd)
            sess.run([complete_optim],feed_dict={z:batch_z,images:batch_images,mask:mask_})
            c_loss,dloss,gloss=sess.run([complete_loss,d_loss,g_loss],feed_dict={z:batch_z,images:batch_images,mask:mask_})
            if np.mod(counter,500)==0:
              print(counter,c_loss,dloss,gloss)  
              
            if np.mod(counter,500)==0:             
                sample_generated,dl,gl=sess.run([G,d_loss,g_loss],feed_dict={z:sample_z,images:sample_images})
                original_part=np.multiply(sample_images,mask_)
                generated_part=np.multiply(sample_generated,imask_)
                total=np.add(original_part,generated_part)
                save_images(original_part,'input\\')
                save_images(total,'final\\')
                print('[Sample] d_loss: %.8f, g_loss: %.8f' % (dl, gl))
                for i in range(20):
                  uploaded = drive.CreateFile({'parents':[{'id': uploadId}]})
                  uploaded.SetContentFile('input\\'+str(i)+'.jpg')
                  uploaded.Upload()
                  uploaded = drive.CreateFile({'parents':[{'id': uploadId}]})
                  uploaded.SetContentFile('final\\'+str(i)+'.jpg')
                  uploaded.Upload()
                   
                
            if np.mod(counter,5000)==0:
                modelName='ModelEpoch'+str(counter)
                chkName = modelName+'.ckpt'
                save_path = saver.save(sess, "data/"+chkName)
                print(counter,' : done saving at',save_path)
                print(os.getcwd())
                metaName=chkName+".meta"
                check="data/"+metaName
                uploaded = drive.CreateFile({'parents':[{'id': uploadId}] , 'title' : metaName})
                uploaded.SetContentFile(check)
                uploaded.Upload()
                dataName=chkName+".data-00000-of-00001"
                check="data/"+dataName
                uploaded = drive.CreateFile({'parents':[{'id': uploadId}] , 'title' : dataName})
                uploaded.SetContentFile(check)
                uploaded.Upload()
                indexName=chkName+".index"
                check="data/"+indexName
                uploaded = drive.CreateFile({'parents':[{'id': uploadId}] , 'title' : indexName})
                uploaded.SetContentFile(check)
                uploaded.Upload()
            counter+=1


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


500 288752.47 0.03472829 4.2817307
[Sample] d_loss: 0.12909032, g_loss: 4.49697447


ApiRequestError: ignored

## Saving the model

In [31]:
!ls

datalab       image\3.jpg	    ops.py	    samples\39.jpg
image\0.jpg   image\40.jpg	    __pycache__     samples\3.jpg
image\10.jpg  image\41.jpg	    samples\0.jpg   samples\40.jpg
image\11.jpg  image\42.jpg	    samples\10.jpg  samples\41.jpg
image\12.jpg  image\43.jpg	    samples\11.jpg  samples\42.jpg
image\13.jpg  image\44.jpg	    samples\12.jpg  samples\43.jpg
image\14.jpg  image\45.jpg	    samples\13.jpg  samples\44.jpg
image\15.jpg  image\46.jpg	    samples\14.jpg  samples\45.jpg
image\16.jpg  image\47.jpg	    samples\15.jpg  samples\46.jpg
image\17.jpg  image\48.jpg	    samples\16.jpg  samples\47.jpg
image\18.jpg  image\49.jpg	    samples\17.jpg  samples\48.jpg
image\19.jpg  image\4.jpg	    samples\18.jpg  samples\49.jpg
image\1.jpg   image\50.jpg	    samples\19.jpg  samples\4.jpg
image\20.jpg  image\51.jpg	    samples\1.jpg   samples\50.jpg
image\21.jpg  image\52.jpg	    samples\20.jpg  samples\51.jpg
image\22.jpg  image\53.jpg	    samples\21.jpg  samples\52.jpg
image\23.jpg  ima

In [0]:
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving ModelEpoch5500.data-00000-of-00001 to ModelEpoch5500.data-00000-of-00001
